<a href="https://colab.research.google.com/github/dolDolSee/TF2.0/blob/main/catndog_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
paths=[]
dataset_gubuns=[]
label_gubuns=[]

for dirname, _, filenames in os.walk("/kaggle/input/cat-and-dog"):
    for filename in filenames:
        if ".jpg" in filename:
            file_path = dirname +'/'+filename
            paths.append(file_path)
            if "/training_set/" in file_path:
                dataset_gubuns.append("train")
            elif "/test_set/" in file_path:
                dataset_gubuns.append("test")
            else: 
                dataset_gubuns.append("N/A")
            
            if 'dogs' in file_path:
                label_gubuns.append('DOG')
            elif 'cats' in file_path:
                label_gubuns.append('CAT')
            else: label_gubuns.append('N/A')

In [ ]:
paths[:10] , dataset_gubuns[:10], label_gubuns[:10]

In [ ]:
pd.set_option('display.max_colwidth', 200)

data_df = pd.DataFrame({'path':paths, 'dataset':dataset_gubuns, "label":label_gubuns})
data_df['dataset']

In [ ]:
print(data_df['dataset'].value_counts())
print(data_df['label'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

def show_grid_images(image_path_list, ncols=8, augmentor=None, title=None):
    figure, axs = plt.subplots(figsize=(22,6), nrows = 1, ncols=ncols)
    for i in range(ncols):
        image = cv2.cvtColor(cv2.imread(image_path_list[i]), cv2.COLOR_BGR2RGB)
        axs[i].imshow(image)
        axs[i].set_title(title)

        
dog_image_list = data_df[data_df['label']=="DOG"]['path'].iloc[:6].tolist()
show_grid_images(dog_image_list, ncols=6, title="DOG")

cat_image_list = data_df[data_df['label']=="CAT"]['path'].iloc[:6].tolist()
show_grid_images(cat_image_list, ncols=6, title="CAT")

In [ ]:
for image_path in dog_image_list:
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    print('image shape:', image.shape)
    
print('image height shape:', image[:, 0, 0].shape)
print('image width shape:', image[0, :, 0].shape)
print('image size:', image[:, :, 0].shape)

print('### image array:\n', image)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(horizontal_flip=True, rescale=1/255.)
#flow from directory 잘 안씀 dataFrame쓰는게 편함
train_flow_gen =train_gen.flow_from_directory(directory="/kaggle/input/cat-and-dog/training_set/training_set"
                                    , target_size = (224,224)
                                    ,class_mode = 'categorical'
                                    ,batch_size=64)

In [ ]:
images_array, labels_array = next(train_flow_gen)

In [ ]:
print(images_array.shape, labels_array.shape)

In [ ]:
images_array = next(train_flow_gen)[0]
labels_array = next(train_flow_gen)[1]
print('##### image array shape:', images_array.shape)
print('#### label array shape:', labels_array.shape)

In [ ]:
IMAGE_SIZE = 224
BATCH_SIZE = 64

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Accuracy

class XceptionCD(tf.keras.Model):
    def __init__(self,model_name):
        super(XceptionCD,self).__init__()
        self.xception_layer = model_name(input_shape=(224,224,3), include_top=False, weights="imagenet")
        self.pooling_layer = tf.keras.layers.GlobalAveragePooling2D()
        self.Dropout_layer = tf.keras.layers.Dropout(rate=0.5)
        self.Dense_layer = tf.keras.layers.Dense(50, activation="relu", name="fc1")
        self.output_layer = tf.keras.layers.Dense(2, activation="relu", name='output')
        
    
    def call(self,x):
        input = tf.reshape(x,[-1,224,224,3])
        x = self.xception_layer(input)
        x = self.pooling_layer(x)
        x = self.Dropout_layer(x)
        x = self.Dense_layer(x)
        x = self.output_layer(x)
        return x
model = XceptionCD(model_name=tf.keras.applications.Xception)
model.compile(optimizer=Adam(0.001),loss="categorical_crossentropy",metrics=["Accuracy"])


In [ ]:
model.fit(train_flow_gen, epochs=10)

In [ ]:
test_gen = ImageDataGenerator(rescale=1/255.0)
test_flow_gen = test_gen.flow_from_directory("/kaggle/input/cat-and-dog/test_set/test_set"
                                            , target_size=(224,224)
                                            , class_mode = 'categorical'
                                            , batch_size = 64, shuffle=False)
model.evaluate(test_flow_gen)

flow_from_dataframe

In [ ]:
model.summary()

In [ ]:
data_df = pd.DataFrame({'path':paths, 'dataset':dataset_gubuns,'label':label_gubuns})
print(data_df['dataset'].value_counts())
data_df.head(10)

In [ ]:
train_df = data_df[data_df['dataset']=='train']
test_df = data_df[data_df['dataset']=='test']
print(train_df.shape, test_df.shape)

In [ ]:
from sklearn.model_selection import train_test_split

tr_df, val_df= train_test_split(train_df, test_size=0.15, stratify=train_df['label'], random_state=2021)
print(tr_df.shape, val_df.shape)

In [ ]:
IMAGE_SIZE=224
BATCH_SIZE=64

In [ ]:
tr_generator = ImageDataGenerator(horizontal_flip=True, rescale=1/255.0)
tr_flow_gen = tr_generator.flow_from_dataframe(dataframe=tr_df
                                              ,x_col='path'
                                              ,y_col='label'
                                              ,target_size=(IMAGE_SIZE,IMAGE_SIZE)
                                              ,class_mode = 'binary'
                                              ,batch_size=BATCH_SIZE
                                              ,shuffle=True)


In [ ]:
image_array , labels_array = next(tr_flow_gen)
print(image_array.shape, labels_array.shape)

In [ ]:
val_generator = ImageDataGenerator(rescale=1/255.)
val_flow_gen = val_generator.flow_from_dataframe(dataframe=val_df
                                                ,x_col='path'
                                                ,y_col='label'
                                                ,target_size=(IMAGE_SIZE,IMAGE_SIZE)
                                                ,class_mode = 'binary'
                                                ,batch_size=BATCH_SIZE
                                                ,suffle=False)

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint , LearningRateScheduler

class XceptionCD(tf.keras.Model):
    def __init__(self,model_name):
        super(XceptionCD,self).__init__()
        self.xception_layer = model_name(input_shape=(224,224,3), include_top=False, weights="imagenet")
        self.pooling_layer = tf.keras.layers.GlobalAveragePooling2D()
        self.Dropout_layer = tf.keras.layers.Dropout(rate=0.5)
        self.Dense_layer = tf.keras.layers.Dense(50, activation="relu", name="fc1")
        self.output_layer = tf.keras.layers.Dense(1, activation="sigmoid", name='output')
        
    
    def call(self,x):
        input = tf.reshape(x,[-1,224,224,3])
        x = self.xception_layer(input)
        x = self.pooling_layer(x)
        x = self.Dropout_layer(x)
        x = self.Dense_layer(x)
        x = self.output_layer(x)
        return x
model = XceptionCD(model_name=tf.keras.applications.Xception)
model.compile(optimizer=Adam(0.001),loss="binary_crossentropy",metrics=["Accuracy"])
rlr_cb = ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, mode='min',verbose=1)
ely_cb = EarlyStopping(monitor="val_loss", patience=5, mode='min', verbose=1)




In [ ]:
model.fit(tr_flow_gen, epochs=15, validation_data=val_flow_gen, callbacks=[rlr_cb, ely_cb])

In [ ]:
test_generator = ImageDataGenerator(rescale=1/255.)
test_flow_gen = test_generator.flow_from_dataframe(dataframe=test_df
                                                  ,x_col='path'
                                                  ,y_col='label'
                                                  ,target_size=(IMAGE_SIZE,IMAGE_SIZE)
                                                  ,class_mode='binary'
                                                  ,batch_size = BATCH_SIZE
                                                  ,shuffle=False)
model.evaluate(test_flow_gen)


torch? tf? preprocessing ways

In [ ]:
from tensorflow.keras.applications.xception import preprocess_input

#rescale 대신에 preprocess_function에 preprocess_input으로 input rescale해서 넣어줌.
tr_generator = ImageDataGenerator(horizontal_flip=True, preprocessing_function=preprocess_input)
tr_flow_gen = tr_generator.flow_from_dataframe(dataframe=tr_df
                                              ,xcol="path"
                                              ,ycol="label"
                                              ,target_size=(IMAGE_SIZE,IMAGE_SIZE)
                                              ,class_mode='binary'
                                              ,batch_size=BATCH_SIZE
                                              ,shuffle=True)